# [Direct Preference Optimization: Your Language Model is Secretly a Reward Model (DPO)](https://arxiv.org/pdf/2305.18290.pdf)

### Reference Code 
- https://huggingface.co/docs/trl/main/en/dpo_trainer
- https://github.com/huggingface/trl/blob/main/examples/scripts/dpo.py

Therefore the final dataset object should contain these 3 entries if you use the default DPODataCollatorWithPadding data collator. 

The entries should be named:
- prompt
- chosen
- rejected

In [24]:
import os
import torch
# Set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [25]:
# dpo_dataset_dict = {
#     "prompt": [
#         "hello",
#         "how are you",
#         "What is your name?",
#         "What is your name?",
#         "Which is the best programming language?",
#         "Which is the best programming language?",
#         "Which is the best programming language?",
#     ],
#     "chosen": [
#         "hi nice to meet you",
#         "I am fine",
#         "My name is Mary",
#         "My name is Mary",
#         "Python",
#         "Python",
#         "Java",
#     ],
#     "rejected": [
#         "leave me alone",
#         "I am not fine",
#         "Whats it to you?",
#         "I dont have a name",
#         "Javascript",
#         "C++",
#         "C++",
#     ],
# }

In [26]:
from datasets import load_dataset

dpo_dataset_dict = load_dataset("argilla/ultrafeedback-binarized-preferences-cleaned")

In [27]:
dpo_dataset_dict = dpo_dataset_dict['train'].select_columns(['prompt', 'chosen', 'rejected'])
dpo_dataset_dict

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 60917
})

In [28]:
dpo_dataset_dict['prompt'][1]

'Suppose you are a content creator and want to generate compelling titles and descriptions for your YouTube videos automatically. You have decided to use GPT to solve this problem.\nCan you provide an example of how you would use GPT to generate titles and descriptions for your YouTube videos automatically? Please include the necessary steps and any additional information that may be helpful.'

In [29]:
# !pip install trl
# !pip install tf_keras
# !pip install peft

In [30]:
import torch
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    HfArgumentParser, 
    TrainingArguments
)

from typing import Dict, Optional
from trl import DPOTrainer,DPOConfig
# from dpo-qlora-4bit import DPOTrainer
# from dpo import DPOTrainer

# 1. load a pretrained model and tokenizer

In [31]:
model_name_or_path = "gpt2"
ignore_bias_buffers = False

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
if ignore_bias_buffers:
    # torch distributed hack
    model._ddp_params_and_buffers_to_ignore = [
        name for name, buffer in model.named_buffers() if buffer.dtype == torch.bool
    ]

ref_model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

The DPO trainer expects a model of AutoModelForCausalLM, compared to PPO that expects AutoModelForCausalLMWithValueHead for the value function.

## 2. Load the Anthropic Helpful-Harmless dataset

In [32]:
def extract_anthropic_prompt(prompt_and_response):
    """Extract the anthropic prompt from a prompt and response pair."""
    search_term = "\n\nAssistant:"
    search_term_idx = prompt_and_response.rfind(search_term)
    assert search_term_idx != -1, f"Prompt and response does not contain '{search_term}'"
    return prompt_and_response[: search_term_idx + len(search_term)]

def get_hh(split: str, sanity_check: bool = False, silent: bool = False, cache_dir: str = None) -> Dataset:
    """Load the Anthropic Helpful-Harmless dataset from Hugging Face and convert it to the necessary format.

    The dataset is converted to a dictionary with the following structure:
    {
        'prompt': List[str],
        'chosen': List[str],
        'rejected': List[str],
    }

    Prompts should be structured as follows:
      \n\nHuman: <prompt>\n\nAssistant:
    Multiple turns are allowed, but the prompt should always start with \n\nHuman: and end with \n\nAssistant:.
    """

    dataset = load_dataset("Anthropic/hh-rlhf", split=split, cache_dir=cache_dir)
    if sanity_check:
        dataset = dataset.select(range(min(len(dataset), 1000)))

    def split_prompt_and_responses(sample) -> Dict[str, str]:
        prompt = extract_anthropic_prompt(sample["chosen"])
        return {
            "prompt": prompt,
            "chosen": sample["chosen"][len(prompt) :],
            "rejected": sample["rejected"][len(prompt) :],
        }

    return dataset.map(split_prompt_and_responses)

In [33]:
sanity_check = True
train_dataset = get_hh("train", sanity_check=sanity_check)
eval_dataset = get_hh("test", sanity_check=sanity_check)

In [34]:
train_dataset

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 1000
})

In [35]:
eval_dataset

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 1000
})

# 3. initialize training arguments:

In [36]:
learning_rate = 1e-3
per_device_train_batch_size = 8
gradient_accumulation_steps = 1
max_length= 128
max_prompt_length = 128 
max_target_length =128 
label_pad_token_id = 100
max_steps = 200
# instrumentation
sanity_check = True
report_to = None
gradient_checkpointing = None
beta = 0.1

In [ ]:
training_args = DPOConfig(
    per_device_train_batch_size=per_device_train_batch_size,
    max_steps=max_steps,
    remove_unused_columns=False,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    eval_strategy="steps",
    logging_first_step=True,
    logging_steps=5,  # match results in blog post
    eval_steps=500,
    output_dir="./test",
    optim="rmsprop",
    warmup_steps=150,
    report_to=report_to,
    bf16=False,
    gradient_checkpointing=gradient_checkpointing,
    # TODO: uncomment that on the next transformers release
    # gradient_checkpointing_kwargs=gradient_checkpointing_kwargs,
)

# 4. initialize the DPO trainer

In [40]:
dpo_trainer = DPOTrainer(
    model,
    ref_model,
    args=training_args,
    beta=beta,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    max_length=max_length,
    max_target_length=max_target_length,
    max_prompt_length=max_prompt_length,
    generate_during_eval=True,
)

/home/jupyter-st124945/.local/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_length, max_target_length, max_prompt_length, generate_during_eval. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in DPOTrainer, please use the DPOConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/jupyter-st124945/.local/lib/python3.12/site-packages/trl/trainer/dpo_trainer.py:397: UserWarning: You passed `generate_during_eval` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
/home/jupyter-st124945/.local/lib/python3.12/site-packages/trl/trainer/dpo_trainer.py:469: UserWarning: You passed `max_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
/home/jupyter-st124945/.local/lib/python3.12/site-packages/trl/trainer/dpo_trainer.py:475: UserWarning: You

In [ ]:
# !pip install trl==0.12
# !pip install wandb
# pip install transformers==4.46
# pip install transformers==4.45.2


In [ ]:
# import trl
# print(trl.__version__)
# import transformers
# print(transformers.__version__)

# 5. Train

In [43]:
print(type(model))  # Should be <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>
print(type(ref_model))  # Should also be a valid Hugging Face model (e.g., GPT2LMHeadModel)


<class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>
<class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>


In [44]:
print(type(model))

<class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>


In [45]:
dpo_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


TrainOutput(global_step=200, training_loss=1.6350819182395935, metrics={'train_runtime': 74.531, 'train_samples_per_second': 21.468, 'train_steps_per_second': 2.683, 'total_flos': 0.0, 'train_loss': 1.6350819182395935, 'epoch': 1.6})

# 5. Save Model

In [48]:
dpo_trainer.save_model()

# 7. Push Model To HuggingFace Hub

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from huggingface_hub import HfApi, HfFolder, Repository
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define repo name (replace with your Hugging Face username and model name)
hf_repo_name = "EkkaratT/a5_DPO_model"  # Change to your repo


model = AutoModelForCausalLM.from_pretrained("./test")
tokenizer = AutoTokenizer.from_pretrained("./test")

# Push model and tokenizer to Hugging Face Hub
model.push_to_hub(hf_repo_name)
tokenizer.push_to_hub(hf_repo_name)

print(f"Model pushed to Hugging Face Hub: https://huggingface.co/{hf_repo_name}")


2025-03-01 05:38:44.757331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740807524.780318 3678504 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740807524.787443 3678504 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-01 05:38:44.815194: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model pushed to Hugging Face Hub: https://huggingface.co/EkkaratT/a5_DPO_model
